In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from sklearn.metrics import mean_squared_error

from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold
from sklearn.model_selection import GridSearchCV

import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Activation
from tensorflow.keras.callbacks import EarlyStopping

In [2]:
total = pd.read_csv("../CSV/train_data.csv")
test = pd.read_csv("../CSV/pred_test.csv")

print("total shape : {}, test shape : {}".format(total.shape, test.shape))

total shape : (368088, 12), test shape : (15120, 12)


In [3]:
def nmae(true_df, pred_df):
    target_idx = true_df.iloc[:,0]
    pred_df = pred_df[pred_df.iloc[:,0].isin(target_idx)]
    pred_df = pred_df.sort_values(by=[pred_df.columns[0]], ascending=[True])
    true_df = true_df.sort_values(by=[true_df.columns[0]], ascending=[True])
    
    true = true_df.iloc[:,1].to_numpy()
    pred = pred_df.iloc[:,1].to_numpy()
    
    score = np.mean((np.abs(true-pred))/true)
    
    return score

In [4]:
train_years = [2013,2014,2015,2016,2017]
val_years = [2018]

train = total[total['year'].isin(train_years)]
val = total[total['year'].isin(val_years)]

# "year", "month", "day", "hour", "weekday", "구분_int", "기온(°C)"
features = ["month", "hour", "구분_int", "기온(°C)"]
# features = ["year", "month", "day", "hour", "weekday", "구분_int", "기온(°C)"]
train_x = train[features]
train_y = train['공급량']

val_x = val[features]
val_y = val['공급량']

X = total[features]
y = total["공급량"]

In [5]:
print(train_x.shape, train_y.shape)
print(val_x.shape, val_y.shape)
print(test.shape)

(306768, 4) (306768,)
(61320, 4) (61320,)
(15120, 12)


In [6]:
model = Sequential()
model.add(Dense(units = 16, input_dim = 4, activation = "relu"))
model.add(Dense(32, activation = "relu"))
model.add(Dense(64, activation = "relu"))
model.add(Dense(32, activation = "relu"))
model.add(Dense(16, activation = "relu"))
model.add(Dense(1))

model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 16)                80        
                                                                 
 dense_1 (Dense)             (None, 32)                544       
                                                                 
 dense_2 (Dense)             (None, 64)                2112      
                                                                 
 dense_3 (Dense)             (None, 32)                2080      
                                                                 
 dense_4 (Dense)             (None, 16)                528       
                                                                 
 dense_5 (Dense)             (None, 1)                 17        
                                                                 
Total params: 5,361
Trainable params: 5,361
Non-trainabl

2021-11-13 23:05:13.069806: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [7]:
model.compile(loss = "mean_squared_error", 
              optimizer = "rmsprop", 
              metrics = ["accuracy"])

early_stopping = EarlyStopping(patience = 30)

hist = model.fit(train_x, train_y, 
                 epochs = 200, 
                 batch_size = 100, 
                 validation_data = (val_x, val_y), 
                 callbacks = [early_stopping])

Epoch 1/200
3068/3068 [==============================] - 4s 995us/step - loss: 604496.0000 - accuracy: 0.0000e+00 - val_loss: 672543.3750 - val_accuracy: 0.0000e+00
Epoch 2/200
3068/3068 [==============================] - 2s 786us/step - loss: 513921.9688 - accuracy: 0.0000e+00 - val_loss: 589800.3750 - val_accuracy: 0.0000e+00
Epoch 3/200
3068/3068 [==============================] - 3s 1ms/step - loss: 285006.9375 - accuracy: 0.0000e+00 - val_loss: 137787.2031 - val_accuracy: 0.0000e+00
Epoch 4/200
3068/3068 [==============================] - 3s 924us/step - loss: 107432.3203 - accuracy: 0.0000e+00 - val_loss: 77698.0391 - val_accuracy: 0.0000e+00
Epoch 5/200
3068/3068 [==============================] - 3s 862us/step - loss: 77384.5312 - accuracy: 0.0000e+00 - val_loss: 79968.6250 - val_accuracy: 0.0000e+00
Epoch 6/200
3068/3068 [==============================] - 3s 1ms/step - loss: 61988.6133 - accuracy: 0.0000e+00 - val_loss: 54309.7461 - val_accuracy: 0.0000e+00
Epoch 7/200
3068/30

3068/3068 [==============================] - 2s 791us/step - loss: 22726.8066 - accuracy: 0.0000e+00 - val_loss: 34989.3633 - val_accuracy: 0.0000e+00
Epoch 52/200
3068/3068 [==============================] - 2s 803us/step - loss: 22491.0801 - accuracy: 0.0000e+00 - val_loss: 26085.2012 - val_accuracy: 0.0000e+00
Epoch 53/200
3068/3068 [==============================] - 2s 792us/step - loss: 22545.1914 - accuracy: 0.0000e+00 - val_loss: 38846.6211 - val_accuracy: 0.0000e+00


In [8]:
model.evaluate(val_x, val_y)

1917/1917 [==============================] - 1s 527us/step - loss: 38846.6289 - accuracy: 0.0000e+00


[38846.62890625, 0.0]

In [9]:
pred = model.predict(val_x)

true_y = pd.DataFrame({"y" : val_y})
pred_y = pd.DataFrame(pred)

true_y.reset_index(drop = True, inplace = True)
true_y.reset_index(inplace = True)
pred_y.reset_index(inplace = True)

print("NMAE : {:.6f}".format(nmae(true_y, pred_y)))

NMAE : 0.545337


In [10]:
pred_y

,index,0
0,0,1961.825684
1,1,1692.296143
2,2,1639.446533
3,3,1667.221191
4,4,1784.266357
...,...,...
61315,61315,583.491394
61316,61316,622.158691
61317,61317,604.609497
61318,61318,582.618713


In [11]:
true_y

,index,y
0,0,1765.008
1,1,1679.186
2,2,1610.885
3,3,1604.123
4,4,1711.506
...,...,...
61315,61315,681.033
61316,61316,669.961
61317,61317,657.941
61318,61318,610.953


In [12]:
submission = pd.read_csv('../CSV/가스공급량 수요예측 모델개발 data/sample_submission.csv')

In [13]:
test_x = test[features]
preds = model.predict(test_x)
submission['공급량'] = preds
submission.to_csv("../CSV/Submission/20_4_features_DeepLearning_model.csv", index = False)